# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [18]:
!cat publications.json

[
    {
        "pub_date": "2021-09-01", 
        "url_slug": "drip-paper", 
        "paper_url": "(internal)",
        "title": "Squeezing the last DRiP: AutoML for cost-constrained Product classification",
        "venue": "Amazon Machine Learning Conference (AMLC 2021)", 
        "citation": "<u>Abhishek Divekar</u>*, Gaurav Manchanda*, Prit Raj, Abhishek Das, Karan Tanwar, Akshay Jagatap, Vinayak Puranik, Jagannathan Srinivasan, Ramakrishna Nalam, Nikhil Rasiwasia (2021). \"Squeezing the last DRiP: AutoML for cost-constrained product classification\" <i>Amazon Machine Learning Conference</i> (Poster, ~30% acceptance-rate)", 
        "excerpt": "This paper is about the number 1. The number 2 is left for future work."
    },
    {
        "pub_date": "2021-09-01", 
        "url_slug": "cpp-benchmark-paper", 
        "paper_url": "(internal)",
        "title": "The CPP Multimodal AutoML Corpus and Benchmark",
        "venue": "1st Workshop on MultiModal Learning and Fusion, AMLC 2021

## Import pandas

We are using the very handy pandas library for dataframes.

In [19]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [20]:
publications = pd.read_json("publications.json")
publications


,pub_date,url_slug,paper_url,title,venue,citation,excerpt
0,2021-09-01,drip-paper,(internal),Squeezing the last DRiP: AutoML for cost-const...,Amazon Machine Learning Conference (AMLC 2021),"<u>Abhishek Divekar</u>*, Gaurav Manchanda*, P...",This paper is about the number 1. The number 2...
1,2021-09-01,cpp-benchmark-paper,(internal),The CPP Multimodal AutoML Corpus and Benchmark,1st Workshop on MultiModal Learning and Fusion...,"Andrew Borthwick, <u>Abhishek Divekar</u>, Nic...",This paper is about the number 1. The number 2...
2,2021-03-01,leap-paper,(internal),LEAP: LEAf node Predictions in the wild,"2nd ASCS Applied Science Workshop, 2021","<u>Abhishek Divekar</u>, Vinayak Puranik, Zhen...",This paper is about the number 1. The number 2...
3,2020-09-01,eps-paper,(internal),"EPS: a configurable, end-to-end AutoML system","Workshop on Automated Machine Learning, AMLC 2020","Gaurav Manchanda*, <u>Abhishek Divekar</u>*, A...",This paper is about the number 1. The number 2...
4,2018-10-25,kddcup99-paper,https://arxiv.org/abs/1811.05372,Benchmarking datasets for Anomaly-based Networ...,"IEEE International Conference on Computing, Co...","<u>Abhishek Divekar</u>, Meet Parekh, Vaibhav ...",This paper is about the number 1. The number 2...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [21]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [22]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [23]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md 2021-03-01-leap-paper.md
2010-10-01-paper-title-number-2.md 2021-09-01-cpp-benchmark-paper.md
2015-10-01-paper-title-number-3.md 2021-09-01-drip-paper.md
2018-10-25-kddcup99-paper.md       2021-09-01-paper-title-number-1.md
2020-09-01-eps-paper.md


In [24]:
!cat ../_publications/2021-03-01-leap-paper.md

---
title: "LEAP: LEAf node Predictions in the wild"
collection: publications
permalink: /publication/2021-03-01-leap-paper
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2021-03-01
venue: '2nd ASCS Applied Science Workshop, 2021'
paperurl: '(internal)'
citation: '<u>Abhishek Divekar</u>, Vinayak Puranik, Zhenyu Shi, Jinmiao Fu, Nikhil Rasiwasia (2021). &quot;LEAP: LEAf node Predictions in the wild&quot; <i>2nd ASCS Applied Science Workshop, 2021</i> (Oral)'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here]((internal))

Recommended citation: <u>Abhishek Divekar</u>, Vinayak Puranik, Zhenyu Shi, Jinmiao Fu, Nikhil Rasiwasia (2021). "LEAP: LEAf node Predictions in the wild" <i>2nd ASCS Applied Science Workshop, 2021</i> (Oral)